In [2]:
import sys
sys.path.append('../')

from envs.GridMaps import *
from utilities.transitionDatasetGeneration import *

In [3]:
shape = "14x14"
NoWalls = False
store = True

if shape == "5x5":
    maze = maze5x5["maze"]
    n_epochs = 15000
    folder_name = "5x5_5_modelv2"
    n_models = 5
    hidden_size = 32

elif shape == "14x14":
    if NoWalls:
        maze = NoWalls_maze14x14["maze"]
        n_epochs = 120000
        folder_name = "NoWalls_5_model"  #to store the models weights and losses
    else:
        maze = maze14x14["maze"]
        n_epochs = 120000
        folder_name = "5_modelv2"
    n_models = 5
    hidden_size = 512
    

train_transitions, val_transitions = transition_generator(maze)

In [4]:
models_arr = []

for i in range(n_models):
    print(f"Training model {i+1}...")

    model = ModelTrainer(train_transitions, val_transitions, n_epochs, hidden_size, rand_seed=True)

    model.fit()
    models_arr.append(model)

Training model 1...


100%|██████████| 120000/120000 [26:20<00:00, 75.90it/s]


Trained successfully!
Final loss: 0.00046036881394684315

Training model 2...


100%|██████████| 120000/120000 [26:00<00:00, 76.92it/s]


Trained successfully!
Final loss: 5.183694156585261e-05

Training model 3...


...

Trained successfully!
Final loss: 0.00013448305253405124

Training model 4...


100%|██████████| 120000/120000 [25:56<00:00, 77.07it/s]


Trained successfully!
Final loss: 6.14277960266918e-05

Training model 5...


100%|██████████| 120000/120000 [26:14<00:00, 76.24it/s]

Trained successfully!
Final loss: 0.0035991747863590717



In [5]:
if store: store_models(models_arr, folder_name)

In [6]:
print('\nEVALUATION:\n')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

arr1 = []
arr1_raw = []
arr2 = []
arr2_raw = []

for model in models_arr:
    # Single predictions
    with torch.no_grad():
        # Move the input data to the GPU
        inp1 = torch.FloatTensor([[5, 3, 3]]).to(device)
        inp2 = torch.FloatTensor([[5, 4, 3]]).to(device)
        
        # Perform inference on the GPU
        resultado1 = model.model(inp1)
        resultado2 = model.model(inp2)
        
        # Move the result back to CPU and convert to numpy
        resultado1 = resultado1.cpu().detach().numpy()
        resultado2 = resultado2.cpu().detach().numpy()

        arr1.append(np.int32(np.round(resultado1[0])))
        arr1_raw.append(resultado1[0])
        arr2.append(np.int32(np.round(resultado2[0])))
        arr2_raw.append(resultado2[0])
        
for elem, elem2 in zip(arr1, arr2):
    print(elem, '         ', elem2)

#whats the mean and the std of the raw values?
print('\n')
print('Mean of raw values:')
print(np.mean(arr1_raw, axis=0))
print(np.mean(arr2_raw, axis=0))
print('\n')
print('Std of raw values:')
print(np.std(arr1_raw, axis=0), '     ', np.linalg.norm(np.std(arr1_raw, axis=0)))
print(np.std(arr2_raw, axis=0), '     ', np.linalg.norm(np.std(arr2_raw, axis=0)))



EVALUATION:

Using device: cuda
[5 4]           [5 5]
[5 4]           [5 5]
[5 4]           [5 5]
[5 4]           [5 5]
[5 4]           [5 5]


Mean of raw values:
[4.9908724 3.9947696]
[4.989822 4.998473]


Std of raw values:
[0.01379063 0.00532158]       0.014781767
[0.01500997 0.00959466]       0.01781451
